# Autograd vs Bmad Jacobians
In this example, we track single particles through different elements and calculate the Jacobian matrix by using PyTorch autograd. We compare the results with bmad_standard, which uses analytic formulas to calculate the Jacobian. We also test functionalities such as element offsets and tilts. As of now, only transverse transformations are supported.

In [1]:
import torch
from pytao import Tao
import time
import os
torch.set_printoptions(precision= 15, sci_mode=True)

In [2]:
from pathlib import Path

def find_repo(path):
    "Find repository root from the path's parents"
    for path in Path(path).parents:
        # Check whether "path/.git" exists and is a directory
        git_dir = path / ".git"
        if git_dir.is_dir():
            return path

import os
nb_dir = os.getcwd()
# Find the repo root where the script is
repo_path = str(find_repo(nb_dir))
repo_path

'/home/jg2347/Repositories/Bmad-X'

In [3]:
# create incoming particle
from bmadx import Particle, M_ELECTRON

coords = [2e-3,3e-3,-3e-3,-1e-3,2e-3,-2e-3]
coords_t = torch.tensor(coords, dtype=torch.double)
p_in = Particle(*coords_t,
                s = torch.tensor(0.0, dtype=torch.double),
                p0c = torch.tensor(4.0e7, dtype=torch.double),
                mc2 = torch.tensor(M_ELECTRON, dtype=torch.double))
p_in

Particle(x=tensor(2.000000000000000e-03, dtype=torch.float64), px=tensor(3.000000000000000e-03, dtype=torch.float64), y=tensor(-3.000000000000000e-03, dtype=torch.float64), py=tensor(-1.000000000000000e-03, dtype=torch.float64), z=tensor(2.000000000000000e-03, dtype=torch.float64), pz=tensor(-2.000000000000000e-03, dtype=torch.float64), s=tensor(0., dtype=torch.float64), p0c=tensor(4.000000000000000e+07, dtype=torch.float64), mc2=tensor( 5.109989500000000e+05, dtype=torch.float64))

# Drift tests

In [4]:
# Create drift
from bmadx import Drift

d = Drift(L=1.0)
d

Drift(L=1.0)

## Drift one particle test

In [5]:
# Outgoing particle:
from bmadx import track_element

p_out = track_element(p_in, d)

# coordinates as tensor:
x_py = torch.hstack(p_out[:6])
x_py

tensor([5.006027114522933e-03, 3.000000000000000e-03, -4.002009038174311e-03,
        -1.000000000000000e-03, 1.994652573892362e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [6]:
# bmad lattice for comparison
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_drift.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out=tao.orbit_at_s(ele=1)

In [7]:
# bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([5.006027114522930e-03, 3.000000000000000e-03, -4.002009038174310e-03,
        -1.000000000000000e-03, 1.994652573892360e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [8]:
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Drift Jacobian test

In [9]:
# autodiff Jacobian
from torch.autograd.functional import jacobian

f_drift = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), d)[:6]

diff_coords = coords_t.clone().detach().requires_grad_(True)

J = jacobian(f_drift, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000000000000000e+00,  1.002018092527393e+00,  0.000000000000000e+00,
         -3.018117694005117e-06,  0.000000000000000e+00, -3.012081458617107e-03],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.018117694005117e-06,  1.000000000000000e+00,
          1.002010044213542e+00,  0.000000000000000e+00,  1.004027152872369e-03],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.012081458617106e-03,  0.000000000000000e+00,
          1.004027152872369e-03,  1.000000000000000e+00,  1.742165247481025e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [10]:
# Bmad analytical Jacobian
drift_tao = tao.matrix(0,1)
mat_tao = torch.tensor(drift_tao['mat6'])
mat_tao

tensor([[ 1.000000000000000e+00,  1.002018092527390e+00,  0.000000000000000e+00,
         -3.018117694005120e-06,  0.000000000000000e+00, -3.012081458617110e-03],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.018117694005120e-06,  1.000000000000000e+00,
          1.002010044213540e+00,  0.000000000000000e+00,  1.004027152872370e-03],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -3.012081458617110e-03,  0.000000000000000e+00,
          1.004027152872370e-03,  1.000000000000000e+00,  1.742165247481030e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [11]:
# is it close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

True

# Quadrupole tests

In [12]:
# Create quad
from bmadx import Quadrupole

q = Quadrupole(L=0.1, K1=10.0)
q

Quadrupole(L=0.1, K1=10.0, NUM_STEPS=1, X_OFFSET=0.0, Y_OFFSET=0.0, TILT=0.0)

## Quadrupole one particle test

In [13]:
# Outgoing particle
p_out = track_element(p_in, q)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.196239719302552e-03, 8.841834264853368e-04, -3.253441973269224e-03,
        -4.100871741572859e-03, 1.999394664225331e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [14]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [15]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.196239719302550e-03, 8.841834264853370e-04, -3.253441973269220e-03,
        -4.100871741572860e-03, 1.999394664225330e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [16]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Quadrupole Jacobian test

In [17]:
# autodiff Jacobian matrix
f_quadrupole = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), q)[:6]

J = jacobian(f_quadrupole, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 9.503167431875498e-01,  9.853541097581728e-02,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00, -1.924858550317723e-04],
        [-9.833834015386563e-01,  9.503167431875498e-01, -0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.149663908944082e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.050519938506054e+00,
          1.018821577510623e-01,  0.000000000000000e+00,  2.569093937337833e-04],
        [-0.000000000000000e+00,  0.000000000000000e+00,  1.016783934355602e+00,
          1.050519938506054e+00,  0.000000000000000e+00,  1.017485822657404e-04],
        [ 8.003290869842023e-05, -1.942507914386516e-04,  1.543324297486644e-04,
          2.595220753974964e-04,  1.000000000000000e+00,  1.756709202142694e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [18]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'])
mat_tao

tensor([[ 9.503167431875500e-01,  9.853541097581731e-02,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00, -1.924858550317720e-04],
        [-9.833834015386560e-01,  9.503167431875500e-01,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.149663908944080e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.050519938506050e+00,
          1.018821577510620e-01,  0.000000000000000e+00,  2.569093937337830e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.016783934355600e+00,
          1.050519938506050e+00,  0.000000000000000e+00,  1.017485822657410e-04],
        [ 8.003290869842020e-05, -1.942507914386520e-04,  1.543324297486640e-04,
          2.595220753974960e-04,  1.000000000000000e+00,  1.756709202142690e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [19]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

True

## Quadrupole offset test

In [20]:
# Create quad with offsetss
q_off = Quadrupole(L = 0.1,
                   K1 = 10.0,
                   X_OFFSET = 1e-3,
                   Y_OFFSET = -2e-3)
q_off

Quadrupole(L=0.1, K1=10.0, NUM_STEPS=1, X_OFFSET=0.001, Y_OFFSET=-0.002, TILT=0.0)

In [21]:
# Outgoing particle no offset
p_out = track_element(p_in, q)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.196239719302552e-03, 8.841834264853368e-04, -3.253441973269224e-03,
        -4.100871741572859e-03, 1.999394664225331e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [22]:
# Outgoing particle offset
p_out = track_element(p_in, q_off)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.245922976115002e-03, 1.867566828023993e-03, -3.152402096257116e-03,
        -2.067303872861656e-03, 1.999538606428305e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [23]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [24]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.245922976115000e-03, 1.867566828023990e-03, -3.152402096257120e-03,
        -2.067303872861660e-03, 1.999538606428300e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [25]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

In [26]:
# autodiff Jacobian matrix
f_quadrupole_off = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), q_off)[:6]

J = jacobian(f_quadrupole_off, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 9.503167431875498e-01,  9.853541097581728e-02,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00, -2.418522933964102e-04],
        [-9.833834015386563e-01,  9.503167431875498e-01, -0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.315328529941612e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.050519938506054e+00,
          1.018821577510623e-01,  0.000000000000000e+00,  1.548230633218972e-04],
        [-0.000000000000000e+00,  0.000000000000000e+00,  1.016783934355602e+00,
          1.050519938506054e+00,  0.000000000000000e+00,  6.794497089254226e-05],
        [ 1.128356584705092e-04, -2.427969275195176e-04,  8.604405680959595e-05,
          1.557223347173730e-04,  1.000000000000000e+00,  1.726852196136135e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [27]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'])
mat_tao

tensor([[ 9.503167431875500e-01,  9.853541097581731e-02,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00, -2.418522933964100e-04],
        [-9.833834015386560e-01,  9.503167431875500e-01,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.315328529941610e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.050519938506050e+00,
          1.018821577510620e-01,  0.000000000000000e+00,  1.548230633218970e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  1.016783934355600e+00,
          1.050519938506050e+00,  0.000000000000000e+00,  6.794497089254231e-05],
        [ 1.128356584705090e-04, -2.427969275195180e-04,  8.604405680959591e-05,
          1.557223347173730e-04,  1.000000000000000e+00,  1.726852196136140e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [28]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

True

## Quadrupole tilt test (transverse rotation)

In [29]:
# Create quadrupole with tilt
q_tilt = Quadrupole(L = 0.1,
                    K1 = 10.0,
                    TILT = 0.3)
q_tilt

Quadrupole(L=0.1, K1=10.0, NUM_STEPS=1, X_OFFSET=0.0, Y_OFFSET=0.0, TILT=0.3)

In [30]:
# Outgoing particle no tilt
p_out = track_element(p_in, q)
x_py = torch.hstack(p_out[:6])
x_py

tensor([2.196239719302552e-03, 8.841834264853368e-04, -3.253441973269224e-03,
        -4.100871741572859e-03, 1.999394664225331e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [31]:
# Outgoing particle tilt
p_out = track_element(p_in, q_tilt)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.300431812756860e-03, 2.982152956166943e-03, -3.286310353406088e-03,
        -4.782331681936468e-03, 1.999046404985869e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [32]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_quad_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [33]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.300431812756860e-03, 2.982152956166940e-03, -3.286310353406090e-03,
        -4.782331681936470e-03, 1.999046404985870e-03, -2.000000000000000e-03],
       dtype=torch.float64)

In [34]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

In [35]:
# autodiff Jacobian matrix
f_quadrupole_tilt = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), q_tilt)[:6]

J = jacobian(f_quadrupole_tilt, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 9.590677079347456e-01,  9.882768970959287e-02, -2.828949002336301e-02,
         -9.448576885006041e-04,  0.000000000000000e+00, -2.987178598581627e-04],
        [-8.087044026377975e-01,  9.590677079347456e-01, -5.646897158716462e-01,
         -2.828949002336301e-02,  0.000000000000000e+00,  5.996905851096495e-05],
        [-2.828949002336301e-02, -9.448576885006041e-04,  1.041768973758858e+00,
          1.015898790172867e-01,  0.000000000000000e+00,  2.923927872640420e-04],
        [-5.646897158716461e-01, -2.828949002336301e-02,  8.421049354547430e-01,
          1.041768973758858e+00,  0.000000000000000e+00,  1.781243721626206e-04],
        [ 2.398790860688291e-05, -3.005205973357010e-04,  2.310403623486719e-04,
          2.950173386467863e-04,  1.000000000000000e+00,  1.827533631132711e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [36]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'])
mat_tao

tensor([[ 9.590677079347460e-01,  9.882768970959289e-02, -2.828949002336300e-02,
         -9.448576885006040e-04,  0.000000000000000e+00, -2.987178598581630e-04],
        [-8.087044026377980e-01,  9.590677079347460e-01, -5.646897158716460e-01,
         -2.828949002336300e-02,  0.000000000000000e+00,  5.996905851096470e-05],
        [-2.828949002336300e-02, -9.448576885006040e-04,  1.041768973758860e+00,
          1.015898790172870e-01,  0.000000000000000e+00,  2.923927872640420e-04],
        [-5.646897158716460e-01, -2.828949002336300e-02,  8.421049354547430e-01,
          1.041768973758860e+00,  0.000000000000000e+00,  1.781243721626220e-04],
        [ 2.398790860688290e-05, -3.005205973357010e-04,  2.310403623486720e-04,
          2.950173386467860e-04,  1.000000000000000e+00,  1.827533631132710e-05],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  1.000000000000000e+00]],
       dtype=torch.fl

In [37]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-14)

True

# Crab_Cavity tests

In [38]:
# Create cavity
from bmadx import CrabCavity

cav = CrabCavity(L=0.2,
                 VOLTAGE=1e4,
                 PHI0=0.5,
                 RF_FREQUENCY=1e9)
cav

CrabCavity(L=0.2, VOLTAGE=10000.0, PHI0=0.5, RF_FREQUENCY=1000000000.0, X_OFFSET=0.0, Y_OFFSET=0.0, TILT=0.0)

## Crab_cavity one particle test

In [39]:
# Outgoing particle
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.600159503009856e-03, 2.989525785200116e-03, -3.200403013811110e-03,
        -1.000000000000000e-03, 1.998933450470348e-03, -2.012044669173099e-03],
       dtype=torch.float64)

In [40]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [41]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.600159503009860e-03, 2.989525785200120e-03, -3.200403013811110e-03,
        -1.000000000000000e-03, 1.998933450470350e-03, -2.012044669173100e-03],
       dtype=torch.float64)

In [42]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## Crab_cavity Jacobian test

In [43]:
# autodiff Jacobian matrix
f_crab_cavity = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), cav)[:6]

J = jacobian(f_crab_cavity, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000001571489388e+00,  2.004051338856653e-01,  0.000000000000000e+00,
         -6.552534052264527e-07, -5.246100940167111e-04, -6.013813103820471e-04],
        [ 0.000000000000000e+00,  1.000001576957398e+00,  0.000000000000000e+00,
         -5.256524659788450e-07, -5.235440739575950e-03, -8.949162872176154e-08],
        [-5.256651057680826e-07, -6.552540421507948e-07,  1.000000000000000e+00,
          2.004032150230133e-01,  2.637597029868803e-09,  2.008080032283205e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00, -0.000000000000000e+00,  0.000000000000000e+00],
        [-9.289874897604998e-08, -6.013816504874850e-04, -0.000000000000000e+00,
          2.008080022802770e-04,  1.000001569699991e+00,  3.483769060576968e-05],
        [-5.235440749927688e-03, -5.246038241196926e-04, -0.000000000000000e+00,
          2.643031443043208e-09,  1.058650459477272e-05,  1.000001579115604e+00]],
       dtype=torch.fl

In [44]:
# Bmad formula Jacobian
cav_tao = tao.matrix(0,1)
mat_tao = torch.tensor(cav_tao['mat6'])
mat_tao

tensor([[ 1.000001571489390e+00,  2.004051338856650e-01,  0.000000000000000e+00,
         -6.552534052264531e-07, -5.246100940167110e-04, -6.013813103820470e-04],
        [ 0.000000000000000e+00,  1.000001576957400e+00,  0.000000000000000e+00,
         -5.256524659788450e-07, -5.235440739575950e-03, -8.949162872175810e-08],
        [-5.256651057680830e-07, -6.552540421507950e-07,  1.000000000000000e+00,
          2.004032150230130e-01,  2.637597029868800e-09,  2.008080032283200e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [-9.289874897604990e-08, -6.013816504874850e-04,  0.000000000000000e+00,
          2.008080022802770e-04,  1.000001569699990e+00,  3.483769060576920e-05],
        [-5.235440749927690e-03, -5.246038241196930e-04,  0.000000000000000e+00,
          2.643031443043210e-09,  1.058650459477270e-05,  1.000001579115600e+00]],
       dtype=torch.fl

In [45]:
# close to Tao result up to 13th decimal place
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

True

In [46]:
# but not up to 14th decimal place
torch.isclose(mat_py, mat_tao, atol=0, rtol=1e-14)

tensor([[ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True,  True]])

## Crab_cavity offset test

In [47]:
# Cavity with offsets
cav_off = CrabCavity(L=0.2,
                     VOLTAGE=1e4,
                     PHI0=0.5,
                     RF_FREQUENCY=1e9,
                     X_OFFSET=1e-3,
                     Y_OFFSET=-2e-3)
cav_off

CrabCavity(L=0.2, VOLTAGE=10000.0, PHI0=0.5, RF_FREQUENCY=1000000000.0, X_OFFSET=0.001, Y_OFFSET=-0.002, TILT=0.0)

In [48]:
# Outgoing particle no offset
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.600159503009856e-03, 2.989525785200116e-03, -3.200403013811110e-03,
        -1.000000000000000e-03, 1.998933450470348e-03, -2.012044669173099e-03],
       dtype=torch.float64)

In [49]:
# Outgoing particle offset
p_out = track_element(p_in, cav_off)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.600157931528713e-03, 2.989525785200116e-03, -3.200402488148762e-03,
        -1.000000000000000e-03, 1.998933543368366e-03, -2.006809228425473e-03],
       dtype=torch.float64)

In [50]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [51]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.600157931528710e-03, 2.989525785200120e-03, -3.200402488148760e-03,
        -1.000000000000000e-03, 1.998933543368370e-03, -2.006809228425470e-03],
       dtype=torch.float64)

In [52]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

In [53]:
# autodiff Jacobian matrix
f_crab_cavity_off = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), cav_off)[:6]

J = jacobian(f_crab_cavity_off, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000001571472898e+00,  2.004046082062692e-01,  0.000000000000000e+00,
         -6.552482567216043e-07, -5.246059606096404e-04, -6.013781607107568e-04],
        [ 0.000000000000000e+00,  1.000001576957398e+00,  0.000000000000000e+00,
         -5.256524659788450e-07, -5.235440739575950e-03, -8.949162872176154e-08],
        [-5.256595900031537e-07, -6.552486167910892e-07,  1.000000000000000e+00,
          2.004026893590357e-01,  2.175539466651803e-09,  2.008069496751906e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00, -0.000000000000000e+00,  0.000000000000000e+00],
        [-9.289728709737649e-08, -6.013785012326269e-04, -0.000000000000000e+00,
          2.008069488994388e-04,  1.000001570461304e+00,  3.483741139091059e-05],
        [-5.235440745428065e-03, -5.246024376227188e-04, -0.000000000000000e+00,
          2.181016073880466e-09,  5.984882352954397e-06,  1.000001578177490e+00]],
       dtype=torch.fl

In [54]:
# Bmad formula Jacobian
cav_tao = tao.matrix(0,1)
mat_tao = torch.tensor(cav_tao['mat6'])
mat_tao

tensor([[ 1.000001571472900e+00,  2.004046082062690e-01,  0.000000000000000e+00,
         -6.552482567216040e-07, -5.246059606096400e-04, -6.013781607107571e-04],
        [ 0.000000000000000e+00,  1.000001576957400e+00,  0.000000000000000e+00,
         -5.256524659788450e-07, -5.235440739575950e-03, -8.949162872175810e-08],
        [-5.256595900031540e-07, -6.552486167910890e-07,  1.000000000000000e+00,
          2.004026893590360e-01,  2.175539466651800e-09,  2.008069496751910e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [-9.289728709737820e-08, -6.013785012326270e-04,  0.000000000000000e+00,
          2.008069488994390e-04,  1.000001570461300e+00,  3.483741139091050e-05],
        [-5.235440745428060e-03, -5.246024376227190e-04,  0.000000000000000e+00,
          2.181016073880470e-09,  5.984882352954400e-06,  1.000001578177490e+00]],
       dtype=torch.fl

In [55]:
# close to Tao result up to 13th decimal place
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

True

## Crab_cavity tilt test

In [56]:
# Cavity with tilts
cav_tilt = CrabCavity(L=0.2,
                      VOLTAGE=1e4,
                      PHI0=0.5,
                      RF_FREQUENCY=1e9,
                      TILT=0.3)
cav_tilt

CrabCavity(L=0.2, VOLTAGE=10000.0, PHI0=0.5, RF_FREQUENCY=1000000000.0, X_OFFSET=0.0, Y_OFFSET=0.0, TILT=0.3)

In [57]:
# Outgoing particle no offset
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.600159503009856e-03, 2.989525785200116e-03, -3.200403013811110e-03,
        -1.000000000000000e-03, 1.998933450470348e-03, -2.012044669173099e-03],
       dtype=torch.float64)

In [58]:
# Outgoing particle offset and tilt
p_out = track_element(p_in, cav_tilt)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.600204778962090e-03, 2.989993600406731e-03, -3.200712636808565e-03,
        -1.003095342122277e-03, 1.998933093429908e-03, -2.006710147677921e-03],
       dtype=torch.float64)

In [59]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_crab_cavity_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [60]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.600204778962090e-03, 2.989993600406730e-03, -3.200712636808560e-03,
        -1.003095342122280e-03, 1.998933093429910e-03, -2.006710147677920e-03],
       dtype=torch.float64)

In [61]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

In [62]:
# autodiff Jacobian matrix
f_crab_cavity_tilt = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), cav_tilt)[:6]

J = jacobian(f_crab_cavity_tilt, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000001501520052e+00,  2.004045849247234e-01,  4.644745816295703e-07,
         -6.073329524777793e-07, -5.011747580192464e-04, -6.014246074929699e-04],
        [ 0.000000000000000e+00,  1.000001506524944e+00,  0.000000000000000e+00,
         -5.021749813760579e-07, -5.001607575171653e-03, -8.549461838918180e-08],
        [-5.037361184290212e-07, -6.073336060452572e-07,  9.999998441761583e-01,
          2.004026493081451e-01, -1.550277786188271e-04,  2.011150437638263e-04],
        [ 0.000000000000000e+00,  4.660227761466018e-07,  0.000000000000000e+00,
          9.999998446590745e-01, -1.547178529322680e-03, -2.644658461431483e-08],
        [-8.875266924563281e-08, -6.014249329362829e-04, -2.745441784728740e-08,
          2.011149416027625e-04,  1.000001344699341e+00,  3.483830346577687e-05],
        [-5.001607580681043e-03, -5.011714382641526e-04, -1.547178531026935e-03,
         -1.550267411292252e-04,  5.897796624455826e-06,  1.000001352386358e+00]],
       dtype=torch.fl

In [63]:
cav_tao = tao.matrix(0,1)
mat_tao = torch.tensor(cav_tao['mat6'])
mat_tao

tensor([[ 1.000001501520050e+00,  2.004045849247230e-01,  4.644745816295700e-07,
         -6.073329524847180e-07, -5.011747580192470e-04, -6.014246074929700e-04],
        [ 0.000000000000000e+00,  1.000001506524940e+00,  0.000000000000000e+00,
         -5.021749813760580e-07, -5.001607575171650e-03, -8.549461838917660e-08],
        [-5.037361184290210e-07, -6.073336060452570e-07,  9.999998441761579e-01,
          2.004026493081450e-01, -1.550277786188270e-04,  2.011150437638260e-04],
        [ 0.000000000000000e+00,  4.660227761466020e-07,  0.000000000000000e+00,
          9.999998446590750e-01, -1.547178529322680e-03, -2.644658461431380e-08],
        [-8.875266924563231e-08, -6.014249329362829e-04, -2.745441784728720e-08,
          2.011149416027630e-04,  1.000001344699340e+00,  3.483830346577600e-05],
        [-5.001607580681040e-03, -5.011714382641520e-04, -1.547178531026930e-03,
         -1.550267411292250e-04,  5.897796624455820e-06,  1.000001352386360e+00]],
       dtype=torch.fl

In [64]:
torch.isclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

tensor([[ True,  True,  True, False,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True]])

In [65]:
torch.isclose(mat_py, mat_tao, atol=0, rtol=1.0e-9)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True]])

In [66]:
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-8)

True

In [67]:
# Something is wrong with the Jacobian matrix of a tilted cavity. Currently debuging.

# RF_Cavity tests

In [68]:
# Create cavity
from bmadx import RFCavity

cav = RFCavity(L=0.2,
               VOLTAGE=1e4,
               PHI0=0.5,
               RF_FREQUENCY=1e9)
cav

RFCavity(L=0.2, VOLTAGE=10000.0, PHI0=0.5, RF_FREQUENCY=1000000000.0, X_OFFSET=0.0, Y_OFFSET=0.0, TILT=0.0)

## RF_Cavity one particle test

In [69]:
# Outgoing particle
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.601208578941774e-03, 3.000000000000000e-03, -3.200402859647258e-03,
        -1.000000000000000e-03, 1.998930328796947e-03, -2.010475072419949e-03],
       dtype=torch.float64)

In [70]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [71]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.601208578941770e-03, 3.000000000000000e-03, -3.200402859647260e-03,
        -1.000000000000000e-03, 1.998930328796950e-03, -2.010475072419950e-03],
       dtype=torch.float64)

In [72]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## RF_Cavity Jacobian test

In [73]:
# autodiff Jacobian matrix
f_rf_cavity = lambda x: track_element(Particle(*x, s=p_in.s, p0c=p_in.p0c, mc2=p_in.mc2), cav)[:6]

J = jacobian(f_rf_cavity, diff_coords)
mat_py = torch.vstack(J)
mat_py

tensor([[ 1.000000000000000e+00,  2.004046705463930e-01,  0.000000000000000e+00,
         -6.036330450442983e-07,  1.577102406617731e-06, -6.024226175949689e-04],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.036330450442982e-07,  1.000000000000000e+00,
          2.004030608582729e-01, -5.257008022059102e-07,  2.008075391983230e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.024226155237540e-04,  0.000000000000000e+00,
          2.008075385079180e-04,  9.999999053429351e-01,  3.484386392945851e-05],
        [ 0.000000000000000e+00,  1.577103892075475e-06,  0.000000000000000e+00,
         -5.257012973584918e-07, -5.235869418127461e-03,  9.999999122191349e-01]],
       dtype=torch.fl

In [74]:
# Bmad formula Jacobian
quad_tao = tao.matrix(0,1)
mat_tao = torch.tensor(quad_tao['mat6'])
mat_tao

tensor([[ 1.000000000000000e+00,  2.004046705463930e-01,  0.000000000000000e+00,
         -6.036330450442980e-07,  1.577102406617730e-06, -6.024226175949690e-04],
        [ 0.000000000000000e+00,  1.000000000000000e+00,  0.000000000000000e+00,
          0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.036330450442980e-07,  1.000000000000000e+00,
          2.004030608582730e-01, -5.257008022059100e-07,  2.008075391983230e-04],
        [ 0.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00,
          1.000000000000000e+00,  0.000000000000000e+00,  0.000000000000000e+00],
        [ 0.000000000000000e+00, -6.024226155237540e-04,  0.000000000000000e+00,
          2.008075385079180e-04,  9.999999053429350e-01,  3.484386392945810e-05],
        [ 0.000000000000000e+00,  1.577103892075480e-06,  0.000000000000000e+00,
         -5.257012973584920e-07, -5.235869418127460e-03,  9.999999122191350e-01]],
       dtype=torch.fl

In [75]:
# close to Tao result?
torch.allclose(mat_py, mat_tao, atol=0, rtol=1.0e-13)

True

## RF_Cavity offset test

In [76]:
# RF cav with offsets
cav_off = RFCavity(L=0.2,
                   VOLTAGE = 1e4,
                   PHI0 = 0.5,
                   RF_FREQUENCY = 1e9,
                   X_OFFSET = 1e-3,
                   Y_OFFSET = -2e-3)
cav_off

RFCavity(L=0.2, VOLTAGE=10000.0, PHI0=0.5, RF_FREQUENCY=1000000000.0, X_OFFSET=0.001, Y_OFFSET=-0.002, TILT=0.0)

In [77]:
# Outgoing particle no offset
p_out = track_element(p_in, cav)

x_py = torch.hstack(p_out[:6])
x_py

tensor([2.601208578941774e-03, 3.000000000000000e-03, -3.200402859647258e-03,
        -1.000000000000000e-03, 1.998930328796947e-03, -2.010475072419949e-03],
       dtype=torch.float64)

In [78]:
# Outgoing particle offset
p_out = track_element(p_in, cav_off)
x_py = torch.hstack(p_out[:6])
x_py

tensor([2.601208578941774e-03, 3.000000000000000e-03, -3.200402859647258e-03,
        -1.000000000000000e-03, 1.998930328796947e-03, -2.010475072419949e-03],
       dtype=torch.float64)

In [79]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity_offset.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [80]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.601208578941770e-03, 3.000000000000000e-03, -3.200402859647260e-03,
        -1.000000000000000e-03, 1.998930328796950e-03, -2.010475072419950e-03],
       dtype=torch.float64)

In [81]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True

## RF_Cavity tilt test

In [82]:
# RF cav with tilt
cav_tilt = RFCavity(L=0.2,
                    VOLTAGE=1e4,
                    PHI0=0.5,
                    RF_FREQUENCY=1e9,
                    TILT=0.3)
cav_tilt

RFCavity(L=0.2, VOLTAGE=10000.0, PHI0=0.5, RF_FREQUENCY=1000000000.0, X_OFFSET=0.0, Y_OFFSET=0.0, TILT=0.3)

In [83]:
# Outgoing particle no offset
p_out = track_element(p_in, cav)
x_py = torch.hstack(p_out[:6])
x_py

tensor([2.601208578941774e-03, 3.000000000000000e-03, -3.200402859647258e-03,
        -1.000000000000000e-03, 1.998930328796947e-03, -2.010475072419949e-03],
       dtype=torch.float64)

In [84]:
# Outgoing particle tilt
p_out = track_element(p_in, cav_tilt)
x_py = torch.hstack(p_out[:6])
x_py

tensor([2.601208578941774e-03, 3.000000000000000e-03, -3.200402859647258e-03,
        -1.000000000000000e-03, 1.998930328796947e-03, -2.010475072419949e-03],
       dtype=torch.float64)

In [85]:
# Bmad lattice to compare
tao = Tao('-lat '+repo_path+'/tests/bmad_lattices/test_rf_cavity_tilt.bmad -noplot')
tao.cmd('set particle_start x='+str(coords[0]))
tao.cmd('set particle_start px='+str(coords[1]))
tao.cmd('set particle_start y='+str(coords[2]))
tao.cmd('set particle_start py='+str(coords[3]))
tao.cmd('set particle_start z='+str(coords[4]))
tao.cmd('set particle_start pz='+str(coords[5]))
orbit_out = tao.orbit_at_s(ele=1)

In [86]:
# Bmad outgoing particle
x_tao = torch.tensor([orbit_out['x'],
                      orbit_out['px'],
                      orbit_out['y'],
                      orbit_out['py'],
                      orbit_out['z'],
                      orbit_out['pz']], dtype=torch.double)
x_tao

tensor([2.601208578941770e-03, 3.000000000000000e-03, -3.200402859647260e-03,
        -1.000000000000000e-03, 1.998930328796950e-03, -2.010475072419950e-03],
       dtype=torch.float64)

In [87]:
# close to Tao result?
torch.allclose(x_py, x_tao, atol=0, rtol=1.0e-14)

True